## Como visualizar filtros e mapas de recursos em redes neurais convolucionais

Redes neurais de aprendizado profundo são geralmente opacas, o que significa que embora possam fazer previsões úteis e habilidosas, não está claro como ou por que uma determinada previsão foi feita.

As redes neurais convolucionais têm estruturas internas projetadas para operar sobre dados de imagem bidimensionais e, como tal, preservam as relações espaciais para o que foi aprendido pelo modelo. Especificamente, os filtros bidimensionais aprendidos pelo modelo podem ser inspecionados e visualizados para descobrir os tipos de recursos que o modelo detectará, e a saída dos mapas de ativação por camadas convolucionais pode ser inspecionada para entender exatamente quais recursos foram detectados para uma determinada entrada imagem.

Neste tutorial, você descobrirá como desenvolver visualizações simples para filtros e mapas de recursos em uma rede neural convolucional.

Depois de concluir este tutorial, você saberá:

Como desenvolver uma visualização para filtros específicos em uma rede neural convolucional.
Como desenvolver uma visualização para mapas de recursos específicos em uma rede neural convolucional.
Como visualizar sistematicamente mapas de recursos para cada bloco em uma rede neural convolucional profunda.

## Visão geral do tutorial
Este tutorial é dividido em quatro partes; eles são:

1. Visualizando Camadas Convolucionais
2. Modelo VGG de pré-ajuste
3. Como visualizar filtros
4. Como visualizar mapas de recursos

## Visualizando Camadas Convolucionais
Os modelos de rede neural são geralmente chamados de opacos. Isso significa que eles não sabem explicar o motivo pelo qual uma decisão ou previsão específica foi feita.

As redes neurais convolucionais são projetadas para trabalhar com dados de imagem, e sua estrutura e função sugerem que devem ser menos inescrutáveis ​​do que outros tipos de redes neurais.

Especificamente, os modelos são compostos de pequenos filtros lineares e o resultado da aplicação de filtros chamados mapas de ativação, ou mais geralmente, mapas de características (ou de atributos).

Filtros e mapas de recursos podem ser visualizados.

Por exemplo, podemos projetar e entender pequenos filtros, como detectores de linha. Talvez a visualização dos filtros em uma rede neural convolucional aprendida possa fornecer uma visão de como o modelo funciona.

Os mapas de recursos que resultam da aplicação de filtros às imagens de entrada e à saída de mapas de recursos de camadas anteriores podem fornecer uma visão sobre a representação interna que o modelo tem de uma entrada específica em um determinado ponto do modelo.

Exploraremos ambas as abordagens para visualizar uma rede neural convolucional neste tutorial.

## Modelo VGG de pré-ajuste

Precisamos de um modelo para visualizar.

Em vez de ajustar um modelo do zero, podemos usar um modelo de classificação de imagem de última geração pré-ajuste.

Keras fornece muitos exemplos de modelos de classificação de imagens de bom desempenho desenvolvidos por diferentes grupos de pesquisa para o ImageNet Large Scale Visual Recognition Challenge, ou ILSVRC. Um exemplo é o modelo VGG-16 que obteve os melhores resultados na competição de 2014.

Este é um bom modelo a ser usado para visualização porque tem uma estrutura uniforme simples de camadas convolucionais e agrupadas ordenadas em série, é profundo com 16 camadas aprendidas e teve um desempenho muito bom, o que significa que os filtros e mapas de recursos resultantes capturarão recursos úteis . Para obter mais informações sobre este modelo, [clique aqui](https://arxiv.org/abs/1409.1556).

Executar o exemplo carregará os pesos do modelo na memória e imprimirá um resumo do modelo carregado.

Se for a primeira vez que carrega o modelo, os pesos serão baixados da internet e armazenados em seu diretório pessoal. Esses pesos são de aproximadamente 500 megabytes e o download pode demorar um pouco, dependendo da velocidade de sua conexão com a Internet.

Podemos ver que as camadas são bem nomeadas, organizadas em blocos e nomeadas com índices inteiros dentro de cada bloco.

Podemos carregar e resumir o modelo VGG16 com apenas algumas linhas de código; por exemplo:

In [ ]:
from keras.applications.vgg16 import VGG16
# carrega o modelo
model = VGG16()
# sumário do modelo
model.summary()

## Como visualizar filtros

Talvez a visualização mais simples de realizar seja plotar os filtros aprendidos diretamente.

Na terminologia da rede neural, os filtros aprendidos são simplesmente pesos, mas por causa da estrutura bidimensional especializada dos filtros, os valores de peso têm uma relação espacial entre si e traçar cada filtro como uma imagem bidimensional é significativo (ou poderia ser).

A primeira etapa é revisar os filtros no modelo, para ver com o que temos que trabalhar.

O resumo do modelo impresso na seção anterior resume a forma de saída de cada camada, por exemplo, a forma dos mapas de recursos resultantes. Não dá nenhuma ideia da forma dos filtros (pesos) na rede, apenas o número total de pesos por camada.

Podemos acessar todas as camadas do modelo por meio da propriedade model.layers .

Cada camada possui uma propriedade layer.name , onde as camadas convolucionais têm uma convolução de nomenclatura como o bloco # _conv # , onde o ' # ' é um inteiro. Portanto, podemos verificar o nome de cada camada e pular qualquer uma que não contenha a string ' conv '.

Cada camada convolucional possui dois conjuntos de pesos.

Um é o bloco de filtros e o outro é o bloco de valores de polarização. Eles são acessíveis por meio da função layer.get_weights () . Podemos recuperar esses pesos e resumir sua forma.

O código abaixo imprime uma lista de detalhes da camada, incluindo o nome da camada e a forma dos filtros na camada.

In [ ]:
# summariza os filtros
for layer in model.layers:
	# verifica se é camada convolucional
	if 'conv' not in layer.name:
		continue
	# pega os filtros (pesos)
	filters, biases = layer.get_weights()
	print(layer.name, filters.shape)

Podemos ver que todas as camadas convolucionais usam filtros 3 × 3, que são pequenos e talvez fáceis de interpretar.

Uma preocupação arquitetônica com uma rede neural convolucional é que a profundidade de um filtro deve corresponder à profundidade da entrada do filtro (por exemplo, o número de canais).

Podemos ver que para a imagem de entrada com três canais para vermelho, verde e azul, cada filtro tem uma profundidade de três (aqui estamos trabalhando com um formato de último canal). Podemos visualizar um filtro como um gráfico com três imagens, uma para cada canal, ou compactar todas as três em uma imagem de cor única, ou mesmo apenas olhar para o primeiro canal e assumir que os outros canais terão a mesma aparência. O problema é que temos 63 outros filtros que gostaríamos de visualizar.

Podemos recuperar os filtros da primeira camada convolucional.

Os valores de peso provavelmente serão pequenos valores positivos e negativos centralizados em torno de 0,0.

Podemos normalizar seus valores para o intervalo 0-1 para torná-los fáceis de visualizar.

Podemos enumerar os primeiros seis filtros dos 64 no bloco e representar graficamente cada um dos três canais de cada filtro.

Usamos a biblioteca matplotlib e plotamos cada filtro como uma nova linha de subplots, e cada canal de filtro ou profundidade como uma nova coluna.

O código completo de traçar os primeiros seis filtros da primeira camada convolucional oculta no modelo VGG16 está listado abaixo.

In [ ]:
from matplotlib import pyplot
# recupera os filtros (pesos) da segunda camada (primeira camada convolucional)
filters, biases = model.layers[1].get_weights()
# normaliza para o intervalo [0,1] para torná-los fáceis de visualizar
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plota os primeiros filtros
pyplot.figure(figsize=(10, 10))
n_filters, ix = 6, 1
for i in range(n_filters):
	# pega o filtro
	f = filters[:, :, :, i]
	# plota cada canal separadamente
	for j in range(3):
		ax = pyplot.subplot(n_filters, 3, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plota o canal do filtro em nível de cinza
		pyplot.imshow(f[:, :, j], cmap='gray')
		ix += 1
# mostra a figura
pyplot.show()

Temos então seis linhas de três imagens, ou 18 imagens, uma linha para cada filtro e uma coluna para cada canal

Podemos ver que em alguns casos, o filtro é o mesmo em todos os canais (a primeira linha), e em outros, os filtros são diferentes (a última linha).

Os quadrados escuros indicam pesos pequenos ou inibitórios e os quadrados claros representam pesos grandes ou excitatórios. Usando essa intuição, podemos ver que os filtros na primeira linha detectam um gradiente de luz no canto superior esquerdo para escuro no canto inferior direito.

Embora tenhamos uma visualização, vemos apenas os primeiros seis dos 64 filtros na primeira camada convolucional. É possível visualizar todos os 64 filtros em uma imagem.

Infelizmente, isso não muda; se desejarmos começar a olhar para os filtros na segunda camada convolucional, podemos ver que novamente temos 64 filtros, mas cada um tem 64 canais para corresponder aos mapas de recursos de entrada. Para ver todos os 64 canais em uma linha para todos os 64 filtros, seria necessário (64 × 64) 4.096 subtramas nas quais pode ser difícil ver qualquer detalhe.


## Como visualizar mapas de recursos

Os mapas de ativação, chamados de mapas de recursos, capturam o resultado da aplicação dos filtros à entrada, como a imagem de entrada ou outro mapa de recursos.

A ideia de visualizar um mapa de recursos para uma imagem de entrada específica seria entender quais recursos da entrada são detectados ou preservados nos mapas de recursos. A expectativa seria que os mapas de recursos próximos à entrada detectassem detalhes pequenos ou refinados, enquanto os mapas de recursos próximos à saída do modelo capturassem recursos mais gerais.

Para explorar a visualização de mapas de características, precisamos de dados para o modelo VGG16 que pode ser usado para criar ativações. Usaremos uma simples fotografia de um gato.

In [ ]:
from IPython.display import Image
Image(filename = 'cat.1700.jpg')

Em seguida, precisamos de uma ideia mais clara da forma da saída dos mapas de feições por cada uma das camadas convolucionais e o número do índice da camada para que possamos recuperar a saída da camada apropriada.

O código abaixo irá enumerar todas as camadas no modelo e imprimir o tamanho de saída ou o tamanho do mapa de recursos para cada camada convolucional, bem como o índice de camada no modelo. è possível ver as mesmas formas de saída que vimos no resumo do modelo, mas neste caso apenas para as camadas convolucionais.

In [ ]:
# summarize as formas dos mapas de atributos
for i in range(len(model.layers)):
	layer = model.layers[i]
	# verifica se é uma camada convolucional
	if 'conv' not in layer.name:
		continue
	# sumariza a forma apenas das camadas convolucionais
	print(i, layer.name, layer.output.shape)

Podemos usar essas informações e projetar um novo modelo que é um subconjunto das camadas do modelo VGG16 completo. O modelo teria a mesma camada de entrada do modelo original, mas a saída seria a saída de uma determinada camada convolucional, que sabemos que seria a ativação da camada ou do mapa de feições.

Por exemplo, depois de carregar o modelo VGG, podemos definir um novo modelo que produz um mapa de características da primeira camada convolucional (índice 1) da seguinte maneira:

In [ ]:
from keras.models import Model
# redefine o modelo para a saída logo após a primeira camada oculta
model = Model(inputs=model.inputs, outputs=model.layers[1].output)

Fazer uma previsão com este modelo fornecerá o mapa de recursos para a primeira camada convolucional para uma determinada imagem de entrada fornecida. Vamos implementar isso.

Após definir o modelo, precisamos carregar a imagem do gato com o tamanho esperado pelo modelo, neste caso, 224 × 224.

In [ ]:
from keras.preprocessing.image import load_img
# carrega a imagem com o formato requerido
img = load_img('cat.1700.jpg', target_size=(224, 224))

Em seguida, o objeto [PIL](https://pt.wikipedia.org/wiki/Python_Imaging_Library) de imagem precisa ser convertido em um array NumPy de dados de pixel e expandido de um array 3D para um array 4D com as dimensões de [ samples, rows, cols, channels ], onde temos apenas uma amostra.

In [ ]:
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
# converte a imagem para um array
img = img_to_array(img)
# expande as dimensões de modo que ele represente a uma única amostra
img = expand_dims(img, axis=0)

Os valores de pixel, então, precisam ser dimensionados apropriadamente para o modelo VGG.

In [ ]:
from keras.applications.vgg16 import preprocess_input
# prepara a imagem (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

Agora estamos prontos para obter o mapa de recursos. Podemos fazer isso facilmente chamando a função model.predict () e passando a única imagem preparada.

In [ ]:
# pega o mapa de atributos para a primeira camada escondida
feature_maps = model.predict(img)

Sabemos que o resultado será um mapa de recursos com 224x224x64. Podemos plotar todas as 64 imagens bidimensionais como um quadrado de 8 × 8 de imagens.

In [ ]:
# plota todos os 64 mapas num grid (subplot) 8x8
pyplot.figure(figsize=(10, 10))
square = 8
ix = 1
for _ in range(square):
	for _ in range(square):
		# especifica o subplot e eixo
		ax = pyplot.subplot(square, square, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plota o canal do filtro 
		pyplot.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
		ix += 1
# mostra a figura
pyplot.show()

A saída acima mostra todos os 64 mapas de recursos.

Podemos ver que o resultado da aplicação dos filtros na primeira camada convolucional são muitas versões da imagem do gato com diferentes características destacadas.

Por exemplo, algumas linhas de destaque, outras focam o plano de fundo ou o primeiro plano.

E lembre-se: este modelo é muito menor que o modelo VGG16, mas ainda usa os mesmos pesos (filtros) na primeira camada convolucional que o modelo VGG16:

In [ ]:
model.summary()

Este é um resultado interessante e geralmente corresponde à nossa expectativa. Poderíamos atualizar o exemplo para plotar os mapas de recursos da saída de outras camadas convolucionais específicas.

Outra abordagem seria coletar a saída dos mapas de recursos de cada bloco do modelo em uma única passagem e, em seguida, criar uma imagem de cada um.

Existem cinco blocos principais na imagem (por exemplo, bloco 1, bloco 2, etc.) que terminam em uma camada de agrupamento. Os índices de camada da última camada convolucional em cada bloco são [2, 5, 9, 13, 17].

Podemos definir um novo modelo que possui várias saídas, uma saída de mapa de recursos para cada uma das últimas camadas convolucionais em cada bloco; por exemplo:

In [ ]:
# recarrega o modelo original
model = VGG16()
# redefine o modelo com múltiplas saídas
ixs = [2, 5, 9, 13, 17]
outputs = [model.layers[i+1].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)
#model.summary()

Fazer uma previsão com este novo modelo resultará em uma lista de mapas de recursos.

In [ ]:
feature_maps = model.predict(img)

Sabemos que o número de mapas de características (por exemplo, profundidade ou número de canais) em camadas mais profundas é muito mais do que 64, como 256 ou 512. No entanto, podemos limitar o número de mapas de características visualizados em 64 para consistência.

In [ ]:
square = 8
block = 0
for fmap in feature_maps:
  block = block + 1
  print('\nBloco',block,':')
  pyplot.figure(figsize=(10, 10))
	# plot all 64 maps in an 8x8 squares
  ix = 1
  for _ in range(square):
    for _ in range(square):
      # specify subplot and turn of axis
      ax = pyplot.subplot(square, square, ix)
      ax.set_xticks([])
      ax.set_yticks([])
			# plot filter channel in grayscale
      pyplot.imshow(fmap[0, :, :, ix-1], cmap='gray')
      ix += 1
	# show the figure
  pyplot.show()

A execução do exemplo resulta em cinco grids que mostram os mapas de recursos dos cinco blocos principais do modelo VGG16.

Podemos ver que os mapas de recursos mais próximos da entrada do modelo capturam muitos detalhes finos na imagem e que, à medida que avançamos mais no modelo, os mapas de recursos mostram cada vez menos detalhes.

Esse padrão era esperado, pois o modelo abstrai os recursos da imagem em conceitos mais gerais que podem ser usados ​​para fazer uma classificação. Embora não esteja claro na imagem final que o modelo viu um gato, geralmente perdemos a capacidade de interpretar esses mapas de recursos mais profundos.

